<a href="https://colab.research.google.com/github/krumeto/sebra_data_playground/blob/main/SEBRA_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
!pip install dateparser

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 288 kB 5.2 MB/s 
     |████████████████████████████████| 749 kB 48.3 MB/s 
  Attempting uninstall: regex
    Found existing installation: regex 2022.6.2
    Uninstalling regex-2022.6.2:
      Successfully uninstalled regex-2022.6.2


In [42]:
import pandas as pd
import dateparser

In [2]:
link_to_data = "https://data.egov.bg/resource/download/zip/ba444b96-6ec3-4be7-8981-92bc33d1a94b"

In [3]:
def data_load_sebra(link_to_data):
    data = pd.read_csv(link_to_data, compression = 'zip', parse_dates = ["REG_DATE", "SETTLEMENT_DATE"])
    return(data)

def lowercase_columns(dataf):
    dataf.columns = dataf.columns.str.lower()
    return(dataf)

def add_year(dataf):
    return(
        dataf.assign(reg_year = dataf['reg_date'].dt.year)
        )
    

def pull_government_periods():
    """A function to pull periods of different governments and prepare the info in a format for left join"""

    # Data pull from wikipedia
    governments = pd.read_html("https://bg.wikipedia.org/wiki/%D0%9F%D1%80%D0%B0%D0%B2%D0%B8%D1%82%D0%B5%D0%BB%D1%81%D1%82%D0%B2%D0%B0_%D0%BD%D0%B0_%D0%91%D1%8A%D0%BB%D0%B3%D0%B0%D1%80%D0%B8%D1%8F",
                              match = "Република България"
                              )[0]

    # better column names              
    governments.columns = ["gov_nr", "government_alias", "government_pm", "government_start_dt", "government_end_dt", "n_days", "party/coalition"]

    # Get only post-communist goverments
    post_com_index = governments.loc[lambda d: d["gov_nr"] == "Република България (от 1990 г.)"].index
    governments = governments.iloc[lambda d: d.index > post_com_index[0]]

    # Convert dates using dateparser
    governments.government_start_dt = governments.government_start_dt.apply(dateparser.parse)
    governments.government_end_dt.iloc[:-1] = governments.government_end_dt.iloc[:-1].apply(dateparser.parse)
    governments.government_end_dt = pd.to_datetime(governments.government_end_dt)

    # Get data in a long format
    # Potentially there is a much cleaner way, but data is small and this works
    long_gov_df = pd.DataFrame(columns = ["date", "government_alias", "government_pm", 'party/coalition'])

    # Iter over all rows
    for ix, row in governments.iterrows():
      interim_df = pd.DataFrame(columns = ["date", 'government_alias', 'government_pm', 'party/coalition'])
      # create a date_range starting from the day after a government starts until the day it ends
      try:
        date_period = pd.date_range(start = row["government_start_dt"] + pd.Timedelta(days=1), end = row["government_end_dt"])
      # except for the NA for the latest government
      except ValueError:
        date_period = pd.date_range(start = row["government_start_dt"] + pd.Timedelta(days=1), end = pd.Timestamp.today())
      interim_df["date"] = date_period
      interim_df["government_alias"] = row["government_alias"]
      interim_df["government_pm"] = row["government_pm"]
      interim_df['party/coalition'] = row['party/coalition']
      long_gov_df = pd.concat([long_gov_df, interim_df])

    return(long_gov_df)


In [62]:
governments = pull_government_periods()

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [67]:
data = (data_load_sebra(link_to_data).
        # Lowercase columns
        pipe(lowercase_columns).
        # Add year as a separate column
        pipe(add_year).
        # Add columns for government during the period
        merge(pull_government_periods(), 
              how='left',
              left_on='reg_date',
              right_on = 'date').
        drop(columns = ['date'])
        )

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [68]:
data.info(show_counts = True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3586569 entries, 0 to 3586568
Data columns (total 20 columns):
 #   Column                Non-Null Count    Dtype         
---  ------                --------------    -----         
 0   settlement_date       3586569 non-null  object        
 1   client_receiver_name  3586569 non-null  object        
 2   client_receiver_acc   3586569 non-null  object        
 3   client_receiver_bic   3586569 non-null  object        
 4   fin_code              3586569 non-null  int64         
 5   amount                3586569 non-null  float64       
 6   currency              3586569 non-null  object        
 7   reason1               3459986 non-null  object        
 8   reason2               2626159 non-null  object        
 9   reg_date              3586569 non-null  datetime64[ns]
 10  reg_no                3586569 non-null  object        
 11  sebra_pay_code        3586569 non-null  int64         
 12  organization          3586569 non-null  ob

In [69]:
data.sample(10)

,settlement_date,client_receiver_name,client_receiver_acc,client_receiver_bic,fin_code,amount,currency,reason1,reason2,reg_date,reg_no,sebra_pay_code,organization,primary_organization,primary_org_code,client_name_hash,reg_year,government_alias,government_pm,party/coalition
2543544,20/2/2019,ЯДКИ ДЕТЕЛИНА ЕООД,BG49RZBB91551088710312,RZBBBGSF,1220080000,449114.89,BGN,РСР,NaN,2019-02-19,0,50,РАЗПЛ А-ИЯ-разходи-СЕЛСКИ РАЙОНИ,ДФ Земеделие-Разплащателна агенция,122,54e941d0d95a408c534a6e0e828a432a0e4df3c8ede096...,2019,Борисов 3,Бойко Борисов(род. 1959),"ГЕРБ, Обединени патриоти"
99585,12/3/2007,ТПБП СОФИЯ,BG83REXI93203301230039,REXIBGSF,550090230,1000000.00,BGN,99030556,0307 03079РЕК.11 090307,2007-09-03,000000860,40,ДОО-Пенсии - РУСО-Софийска област,НОИ ? Държавно обществено осигурява,55,ef0edd72f253874511d277d854bc1487f43b4bba38c51a...,2007,Станишев,Сергей Станишев(род. 1966),"БСП, НДСВ, ДПС"
1149751,1/3/2013,СИ ПИ ЕЙ ООД,BG13UNCR70001520614516,UNCRBGSF,9820040209,271056.69,BGN,ОКОНЧАТЕЛНО,0213 021392ТММ02,2013-02-27,E01844,50,МИЕ - ЕФРР-ОП Конкурентноспособност,НФ - ЕФРР,982,390e984e3481527207e2cf479b270db29a3122a65f0848...,2013,Борисов 1,Бойко Борисов(род. 1959),ГЕРБ[3]
778402,2/5/2011,ВИЯД-ИНЖЕНЕРИНГ ООД,BG52UBBS80021068709820,UBBSBGSF,1220093218,120361.85,BGN,ПЛАЩАНЕ НА СУБСИДИИ,NaN,2011-04-29,0,30,РАЗПЛ А-ИЯ-директни-плащания,"ДФ ""ЗЕМЕДЕЛИЕ""",122,dcb7a7e2b4180f3de2c96496d47484e60361b91952d924...,2011,Борисов 1,Бойко Борисов(род. 1959),ГЕРБ[3]
1595157,2/2/2015,ВЕСАН ООД,BG52UBBS84271010692016,UBBSBGSF,223010684,5816.68,BGN,2-РА ДОСТАВКА НА ГОРИВО,0115 0115 Ф-РА /26.01.15,2015-01-28,000220,10,София ПГХВТ,Министерство на земеделието и горите,22,4874f15f04b7e1a9c1d4423377024966387a07e73faff6...,2015,Борисов 2,Бойко Борисов(род. 1959),"ГЕРБ, Реформаторски блок, АБВ"
2594952,25/4/2019,Физическо лице,BG20UNCR70001523219034,UNCRBGSF,1220093218,5866.36,BGN,ЗЕЛЕНИ 2018,NaN,2019-04-24,0,89,РАЗПЛ А-ИЯ-директни-плащания,ДФ Земеделие-Разплащателна агенция,122,b7adc5c63f3a9ef869480ff4ae9cda2792eabcb30fe3d1...,2019,Борисов 3,Бойко Борисов(род. 1959),"ГЕРБ, Обединени патриоти"
2173715,15/9/2017,ЕГЕО 3 ООД,BG63BPBI79371022265901,BPBIBGSF,222003037,15000.00,BGN,НАЕМ КОРАБ ПО ДОГОВОР,ИРР - ЕИК,2017-12-09,000360,10,Институт по рибни ресурси - Варна,Министерство на земеделието и горите,22,0785e24201d4932bb2ed65db45721d072581b65bacfdfb...,2017,Борисов 3,Бойко Борисов(род. 1959),"ГЕРБ, Обединени патриоти"
2631221,13/6/2019,Физическо лице,BG96STSA93000023354701,STSABGSF,1220093239,5993.14,BGN,МЯРКА 11,NaN,2019-12-06,0,40,РАЗПЛ А-ИЯ-директни-плащания-ЕЗФРСР,ДФ Земеделие-Разплащателна агенция,122,61dfeae44d8a6305876bdf091c1d5de29503b01467b908...,2019,Борисов 3,Бойко Борисов(род. 1959),"ГЕРБ, Обединени патриоти"
747844,16/3/2011,Физическо лице,BG09RZBB91551082603718,RZBBBGSF,1220093218,17427.39,BGN,ПЛАЩАНЕ НА СУБСИДИИ,NaN,2011-03-15,0,30,РАЗПЛ А-ИЯ-директни-плащания,"ДФ ""ЗЕМЕДЕЛИЕ""",122,f57780f7db1023e6754741271f7a7fc621f1e1d200a09d...,2011,Борисов 1,Бойко Борисов(род. 1959),ГЕРБ[3]
2850018,10/4/2020,МТУДП ЕООД,BG16UNCR70001519860384,UNCRBGSF,221003520,9240.00,BGN,НАБЛЮДАТЕЛНА ТЕХНИКА-Д.18/16.03.20,ОПОС 2014-2020,2020-03-30,033010000735,88,900 ДПП Врачански балкан,Министерство на земеделието и горите,22,dc9415583217ab0e257adb55010a4e4d92ec88f38cbb4c...,2020,Борисов 3,Бойко Борисов(род. 1959),"ГЕРБ, Обединени патриоти"


In [17]:
# Only transfers in BGN
data.currency.unique()

array(['BGN'], dtype=object)

## Top individuals per amount transferred

In [10]:
(data.
 loc[(data["client_receiver_name"] == "Физическо лице")].
 groupby(["client_receiver_acc", "client_receiver_bic"])['amount'].
 agg(["sum", 'count']).
 sort_values('sum', ascending = False).
 head(20).
 style.format("{:,.0f}")
 )

,,sum,count
client_receiver_acc,client_receiver_bic,,
BG40BNBG96611000066123,BNBGBGSD,"285,715,372","9,980"
BG95BPBI79421076516501,BPBIBGSF,"262,899,263",19
BG52RZBB91553120008113,RZBBBGSF,"57,932,168",98
BG26RZBB91553100055002,RZBBBGSF,"44,503,471",136
BG86TTBB94001525796131,TTBBBG22,"24,315,169",255
BG27KORP92201000000100,KORPBGSF,"16,489,738",9
BG94BPBI79421022439601,BPBIBGSF,"14,583,000",1
BG28BUIN72201010650432,BUINBGSF,"13,409,529",48
BG34UNCR70001500193116,UNCRBGSF,"12,558,646",168


### What can we say about the cure BG51CECB97905053407101 person?

In [ ]:
(data.
 loc[(data["client_receiver_acc"] == "BG51CECB97905053407101")].
 groupby(["organization", "primary_organization"])['amount'].
 agg(["sum", 'count']).
 sort_values('sum', ascending = False).
 head(20).
 style.format("{:,.0f}")
 )

,,sum,count
organization,primary_organization,,
ДОО-Пенсии - ТП-Кърджали,НОИ ? Държавно обществено осигурява,"638,760,800","1,619"
ДОО-Пенсии - РУСО-Кърджали,НОИ ? Държавно обществено осигурява,"19,856,836",58


In [ ]:
(data.
 loc[(data["client_receiver_acc"] == "BG51CECB97905053407101")].
 groupby(["organization", "primary_organization", "reason1"])['amount'].
 agg(["sum", 'count']).
 sort_values('sum', ascending = False).
 head(30).
 style.format("{:,.0f}")
 )

In [18]:
data.loc[data.reason1 == "PENSII08012022"]

,settlement_date,client_receiver_name,client_receiver_acc,client_receiver_bic,fin_code,amount,currency,reason1,reason2,reg_date,reg_no,sebra_pay_code,organization,primary_organization,primary_org_code,client_name_hash,reg_year
3440262,6/1/2022,ОБЛАСТНА ПОЩЕНСКА СТАНЦИЯ,BG51CECB97905053407101,CECBBGSF,550090098,2070000.0,BGN,PENSII08012022,NaN,2022-05-01,BG2200500264,40,ДОО-Пенсии - ТП-Кърджали,НОИ - ДОО,55,538fc48bde95241e9482b09e94349e77490cf241562ea7...,2022
3440496,7/1/2022,ОБЛАСТНА ПОЩЕНСКА СТАНЦИЯ,BG51CECB97905053407101,CECBBGSF,550090098,1795000.0,BGN,PENSII08012022,NaN,2022-06-01,BG2200600388,40,ДОО-Пенсии - ТП-Кърджали,НОИ - ДОО,55,538fc48bde95241e9482b09e94349e77490cf241562ea7...,2022
3440837,10/1/2022,ОБЛАСТНА ПОЩЕНСКА СТАНЦИЯ,BG51CECB97905053407101,CECBBGSF,550090098,1440000.0,BGN,PENSII08012022,NaN,2022-07-01,BG2200700548,40,ДОО-Пенсии - ТП-Кърджали,НОИ - ДОО,55,538fc48bde95241e9482b09e94349e77490cf241562ea7...,2022
3441557,11/1/2022,ОБЛАСТНА ПОЩЕНСКА СТАНЦИЯ,BG51CECB97905053407101,CECBBGSF,550090098,1358000.0,BGN,PENSII08012022,NaN,2022-10-01,BG2201000517,40,ДОО-Пенсии - ТП-Кърджали,НОИ - ДОО,55,538fc48bde95241e9482b09e94349e77490cf241562ea7...,2022
3441851,12/1/2022,ОБЛАСТНА ПОЩЕНСКА СТАНЦИЯ,BG51CECB97905053407101,CECBBGSF,550090098,1104000.0,BGN,PENSII08012022,NaN,2022-11-01,BG2201100486,40,ДОО-Пенсии - ТП-Кърджали,НОИ - ДОО,55,538fc48bde95241e9482b09e94349e77490cf241562ea7...,2022
3442719,13/1/2022,ОБЛАСТНА ПОЩЕНСКА СТАНЦИЯ,BG51CECB97905053407101,CECBBGSF,550090098,1015000.0,BGN,PENSII08012022,NaN,2022-12-01,BG2201200457,40,ДОО-Пенсии - ТП-Кърджали,НОИ - ДОО,55,538fc48bde95241e9482b09e94349e77490cf241562ea7...,2022
3443069,14/1/2022,ОБЛАСТНА ПОЩЕНСКА СТАНЦИЯ,BG51CECB97905053407101,CECBBGSF,550090098,775000.0,BGN,PENSII08012022,NaN,2022-01-13,BG2201300265,40,ДОО-Пенсии - ТП-Кърджали,НОИ - ДОО,55,538fc48bde95241e9482b09e94349e77490cf241562ea7...,2022
3443778,17/1/2022,ОБЛАСТНА ПОЩЕНСКА СТАНЦИЯ,BG51CECB97905053407101,CECBBGSF,550090098,885000.0,BGN,PENSII08012022,NaN,2022-01-14,BG2201400526,40,ДОО-Пенсии - ТП-Кърджали,НОИ - ДОО,55,538fc48bde95241e9482b09e94349e77490cf241562ea7...,2022


In [ ]:
(data.
 loc[(data["client_receiver_name"] == "Физическо лице")].
 groupby(["client_receiver_acc", "client_receiver_bic", "reg_year"])['amount'].
 agg(["sum", 'count']).
 sort_values('sum', ascending = False).
 head(20).
 style.format("{:,.0f}")
 )

In [ ]:
(data.
 loc[(data["client_receiver_name"].str.contains(" АД")) & (~data["client_receiver_name"].str.contains("ОБЩИНА|ОБЩ|ОБЩЕНА"))].
 groupby(["client_receiver_acc", "client_receiver_bic", "client_receiver_name", "reg_year", "reason1"])['amount'].
 agg(["sum", 'count']).
 sort_values('sum', ascending = False).
 head(20).
 style.format("{:,.0f}")
 )